In [1]:
experimentsPath = "/raid/elhamod/Fish/experiments/" #"/raid/elhamod/CIFAR_HGNN/experiments/" #"/raid/elhamod/Fish/experiments/"
dataPath = "/raid/elhamod/Fish/" # "/raid/elhamod/" #"/raid/elhamod/Fish/"
experimentName = "FISH_phylogeny_from_scratch2" #"Fish_tripletloss_alpha_lr_experiments"
device = 0
detailed_reporting = False

In [2]:
import matplotlib.pyplot as plt
import torch
import sys
import os
from sklearn.metrics import f1_score
import pandas as pd
from tqdm import tqdm
from tqdm.auto import trange
import wandb

import warnings
# warnings.filterwarnings("ignore")

try:
    import wandb
except:
    print('wandb not found')

from myhelpers import config_plots, TrialStatistics
from myhelpers.try_warning import try_running
from HGNN.train import CNN, dataLoader
from myhelpers import cifar_dataLoader
from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
config_plots.global_settings()

experimetnsFileName = "experiments.csv"
WANDB_message="wandb not working"


# For logging to server
try_running(lambda : wandb.login(), WANDB_message)

experimentPathAndName = os.path.join(experimentsPath, experimentName)
# set cuda
if device is not None:
    print("using cuda", device)
    torch.cuda.set_device(device)

else:
    print("using cpu")

# get experiment params
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)

# init experiments file
experimentsFileNameAndPath = os.path.join(experimentsPath, experimetnsFileName)

paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)  
experiment_index = 0

# Loop through experiments
# with progressbar.ProgressBar(max_value=number_of_experiments) as bar:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for experiment_params in config_parser.getExperiments():
        print(experiment_params)
        experimentHash =TrialStatistics.getTrialName(experiment_params)

        # load images 
        if experiment_params['image_path'] == 'cifar-100-python':
            datasetManager = cifar_dataLoader.datasetManager(experimentPathAndName, dataPath)
        else:
            datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)
        datasetManager.updateParams(config_parser.fixPaths(experiment_params))
        train_loader, validation_loader, test_loader = datasetManager.getLoaders()
        architecture = {
            "fine": len(train_loader.dataset.csv_processor.getFineList()),
            "coarse" : len(train_loader.dataset.csv_processor.getCoarseList())
        }

        # Loop through n trials
        for i in trange(experiment_params["numOfTrials"], desc="trial"):
            modelName = getModelName(experiment_params, i)
            trialName = os.path.join(experimentPathAndName, modelName)
            trialHash = TrialStatistics.getTrialName(experiment_params, i)

            row_information = {
                'experimentName': experimentName,
                'modelName': modelName,
                'datasetName': getDatasetName(config_parser.fixPaths(experiment_params)),
                'experimentHash': experimentHash,
                'trialHash': trialHash
            }
            row_information = {**row_information, **experiment_params} 
            print(row_information)

            run = try_running(lambda : wandb.init(project='HGNN', group=experimentName+"-"+experimentHash, name=trialHash, config=row_information), WANDB_message) #, reinit=True

            # Train/Load model
            model = CNN.create_model(architecture, experiment_params, device=device)
            
            from torchsummary import summary

            summary(model, (3, 224, 224))

#             try_running(lambda : wandb.watch(model, log="all"), WANDB_message)

            if os.path.exists(CNN.getModelFile(trialName)):
                print("Model {0} found!".format(trialName))
            else:
                initModelPath = CNN.getInitModelFile(experimentPathAndName)
                if os.path.exists(initModelPath):
                    model.load_state_dict(torch.load(initModelPath))
                    print("Init Model {0} found!".format(initModelPath))
                CNN.trainModel(train_loader, validation_loader, experiment_params, model, trialName, test_loader, device=device, detailed_reporting=detailed_reporting)

            # Add to experiments file
            if os.path.exists(experimentsFileNameAndPath):
                experiments_df = pd.read_csv(experimentsFileNameAndPath)
            else:
                experiments_df = pd.DataFrame()

            record_exists = not (experiments_df[experiments_df['modelName'] == modelName][experiments_df['experimentName'] == experimentName]).empty if not experiments_df.empty else False
            if record_exists:
                experiments_df.drop(experiments_df[experiments_df['modelName'] == modelName][experiments_df['experimentName'] == experimentName].index, inplace = True) 

            experiments_df = experiments_df.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
            experiments_df.to_csv(experimentsFileNameAndPath, header=True, index=False)

            try_running(lambda : run.finish(), WANDB_message)

        bar.update()

        experiment_index = experiment_index + 1



# if __name__ == "__main__":
#     torch.multiprocessing.set_start_method('spawn')
    
#     import argparse

#     parser = argparse.ArgumentParser()
#     parser.add_argument('--cuda', required=True, type=int)
#     parser.add_argument('--experiments', required=True)
#     parser.add_argument('--data', required=True)
#     parser.add_argument('--name', required=True)
#     parser.add_argument('--detailed', required=False, action='store_true')
#     args = parser.parse_args()
#     main(experimentName=args.name, experimentsPath=args.experiments, dataPath=args.data, device=args.cuda, detailed_reporting=args.detailed)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mndhamod (use `wandb login --relogin` to force relogin)
experiment:   0%|          | 0/4 [00:00<?, ?it/s]

using cuda 0
{'image_path': 'Curated4/Easy_50', 'suffix': None, 'img_res': 224, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.1, 'numOfTrials': 1, 'fc_layers': 1, 'pretrained': False, 'epochs': 200, 'patience': -1, 'modelType': 'BB', 'lambda': 0, 'tl_model': 'preResNet', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.1, 'tripletEnabled': False, 'tripletSamples': 2, 'tripletSelector': 'semihard', 'tripletMargin': 0.2, 'phylogeny_loss': False, 'displayName': 'Fish preresnet from scratch no triplet', 'noSpeciesBackprop': False, 'phylogeny_loss_epsilon': 0.03}
Creating datasets...
{'Alosa chrysochloris': 482298, 'Carassius auratus': 1005907, 'Cyprinus carpio': 429083, 'Esox americanus': 496115, 'Gambusia affinis': 617445, 'Lepisosteus osseus': 519445, 'Lepisosteus platostomus': 731608, 'Lepomis auritus': 1002718, 'Lepomis cyanellus': 476361, 'Lepomis gibbosus': 670266, 'Lepomis gulosus': 476359, 'Lepomis humilis': 892772, 'Lepomis 

{'experimentName': 'FISH_phylogeny_from_scratch2', 'modelName': 'models/1cd37747e2b0ec16cddd67a8c84cd7db99c5f07d03ebdf1f808a7085', 'datasetName': 'datasplits/7c7513bdfb4e5577fef1c3ec0fa6452d30d87dbc95f258a7c86dd76b', 'experimentHash': 'b083d5cd134e116c889809982c2162efb7e37cfd8d4ff8b4559ad0d7', 'trialHash': '1cd37747e2b0ec16cddd67a8c84cd7db99c5f07d03ebdf1f808a7085', 'image_path': 'Curated4/Easy_50', 'suffix': None, 'img_res': 224, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.1, 'numOfTrials': 1, 'fc_layers': 1, 'pretrained': False, 'epochs': 200, 'patience': -1, 'modelType': 'BB', 'lambda': 0, 'tl_model': 'preResNet', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.1, 'tripletEnabled': False, 'tripletSamples': 2, 'tripletSelector': 'semihard', 'tripletMargin': 0.2, 'phylogeny_loss': False, 'displayName': 'Fish preresnet from scratch no triplet', 'noSpeciesBackprop': False, 'phylogeny_loss_epsilon': 0.03}


wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


layer4 is not in ['conv1', 'layer1', 'layer2', 'layer3', 'bn', 'relu', 'avgpool']
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]             432
            Conv2d-2         [-1, 16, 224, 224]             432
            Conv2d-3         [-1, 16, 224, 224]             432
            Conv2d-4         [-1, 16, 224, 224]             432
       BatchNorm2d-5         [-1, 16, 224, 224]              32
       BatchNorm2d-6         [-1, 16, 224, 224]              32
       BatchNorm2d-7         [-1, 16, 224, 224]              32
              ReLU-8         [-1, 16, 224, 224]               0
              ReLU-9         [-1, 16, 224, 224]               0
             ReLU-10         [-1, 16, 224, 224]               0
           Conv2d-11         [-1, 16, 224, 224]             256
           Conv2d-12         [-1, 16, 224, 224]             256
           Conv2d-13 



iteration:   0%|          | 0/200 [00:00<?, ?it/s]

    BatchNorm2d-2379        [-1, 128, 112, 112]             256
           ReLU-2380        [-1, 128, 112, 112]               0
           ReLU-2381        [-1, 128, 112, 112]               0
         Conv2d-2382         [-1, 32, 112, 112]           4,096
         Conv2d-2383         [-1, 32, 112, 112]           4,096
    BatchNorm2d-2384         [-1, 32, 112, 112]              64
    BatchNorm2d-2385         [-1, 32, 112, 112]              64
           ReLU-2386         [-1, 32, 112, 112]               0
           ReLU-2387         [-1, 32, 112, 112]               0
         Conv2d-2388         [-1, 32, 112, 112]           9,216
         Conv2d-2389         [-1, 32, 112, 112]           9,216
    BatchNorm2d-2390         [-1, 32, 112, 112]              64
    BatchNorm2d-2391         [-1, 32, 112, 112]              64
           ReLU-2392         [-1, 32, 112, 112]               0
           ReLU-2393         [-1, 32, 112, 112]               0
         Conv2d-2394        [-1, 128, 11

/home/elhamod/melhamodenv3/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


iteration:   0%|          | 0/200 [01:15<?, ?it/s, min_val_loss=inf, train=0.00282, val=0.00148, val_loss=3.64]

iteration:   0%|          | 1/200 [01:27<4:50:41, 87.65s/it, min_val_loss=inf, train=0.00282, val=0.00148, val_loss=3.64]


experiment:   0%|          | 0/4 [01:36<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 23.65 GiB total capacity; 22.63 GiB already allocated; 50.00 MiB free; 22.63 GiB reserved in total by PyTorch)

In [ ]:
print(model)

In [ ]:
torch.cuda.device_count()